In [ ]:
# Predictive modelling on AHPF outcomes data
# (readable, step-by-step version)


from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.inspection import permutation_importance

# ---------- Setup ----------
CANDIDATES = [
    Path("ahpf_outcomes_clean.csv"),
    Path("data/anjana/processed/ahpf_outcomes_clean.csv"),
    Path.cwd() / "ahpf_outcomes_clean.csv",
]
AHPF_FILE = next((p for p in CANDIDATES if p.exists()), None)
if AHPF_FILE is None:
    raise FileNotFoundError("Couldn’t find 'ahpf_outcomes_clean.csv'. "
                            "Place it next to the notebook or under data/anjana/processed/.")

print("Using file:", AHPF_FILE)

OUT_DIR = Path("outputs/models")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Choose which outcome to predict 
TARGET = "Infant and young child mortality rate"
# How many past years of the target to use as features
LAGS = [1, 2]

#1) Load & basic clean 
df = pd.read_csv(AHPF_FILE, low_memory=False)
df["Name"]  = df["Name"].astype(str).str.strip()
df["State"] = df["State"].astype(str).str.strip()
df["Year"]  = pd.to_numeric(df["Year"], errors="coerce")

# Make sure Value is numeric even if it came in with commas or unicode minus signs
df["Value"] = (
    df["Value"].astype(str)
      .str.replace(",", "", regex=False)
      .str.replace("\u2212", "-", regex=False)
      .str.extract(r"([-+]?\d*\.?\d+(e[-+]?\d+)?)", expand=False)[0]
      .astype(float)
)

# Keep only rows we can actually model with
df = df.dropna(subset=["State", "Year", "Name", "Value"])

# Prefer rate/percentage style measures over raw counts (keeps the target scale consistent)
rate_like = {"DecimalOne", "DecimalZero", "PercentageOne", "Proportion", "Rate"}
if "UnitType" in df.columns:
    df = df[df["UnitType"].isin(rate_like)]

# 2) Make a panel: rows = (State, Year), columns = indicators
wide = (
    df.pivot_table(index=["State", "Year"], columns="Name", values="Value", aggfunc="mean")
      .sort_index()
      .dropna(axis=1, how="all")
)

if TARGET not in wide.columns:
    print("A few example columns:", list(wide.columns[:15]))
    raise ValueError(f"Target '{TARGET}' not found. Double-check the exact 'Name' text in your CSV.")

# 3) Add lag features for the target (per state)
def add_lags(panel, cols, lag_years):
    out = panel.copy()
    for col in cols:
        for k in lag_years:
            out[f"{col}_lag{k}"] = out.groupby(level=0)[col].shift(k)
    return out

wide_lag = add_lags(wide, [TARGET], LAGS)

#  4) Train/test split that respects time 
years = wide_lag.index.get_level_values("Year").unique().sort_values()
if len(years) < 5:
    raise ValueError("Need at least ~5 years for a proper temporal split.")

test_years  = years[-2:]   # last 2 years → test
train_years = years[:-2]   # the rest     → train

need_cols = [TARGET] + [f"{TARGET}_lag{k}" for k in LAGS]
train = wide_lag.loc[(slice(None), train_years), :].dropna(subset=need_cols)
test  = wide_lag.loc[(slice(None), test_years),  :].dropna(subset=need_cols)

# Features = all indicators except the target + the lagged targets
feat_now = [c for c in wide.columns if c != TARGET]
feat_lag = [c for c in wide_lag.columns if c.startswith(f"{TARGET}_lag")]
X_train = train[feat_now + feat_lag].copy()
y_train = train[TARGET].copy()
X_test  = test[X_train.columns].copy()
y_test  = test[TARGET].copy()

# Simple missing-value handling: fill with train means
col_means = X_train.mean()
X_train = X_train.fillna(col_means)
X_test  = X_test.fillna(col_means)

# ---------- 5) Fit two models ----------
# a) Linear regression (scaled)
scaler = StandardScaler()
Xtr_scaled = scaler.fit_transform(X_train)
Xte_scaled = scaler.transform(X_test)
lin = LinearRegression().fit(Xtr_scaled, y_train)
pred_lin = lin.predict(Xte_scaled)

# b) Random Forest (solid baseline for tabular data)
rf = RandomForestRegressor(n_estimators=600, random_state=42, n_jobs=-1).fit(X_train, y_train)
pred_rf = rf.predict(X_test)

def report(name, y_true, y_pred):
    r2  = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{name}: R² = {r2:.3f} | MAE = {mae:.3f}")
    return r2, mae

print("\nModel performance (test years):")
r2_lin, mae_lin = report("Linear     ", y_test, pred_lin)
r2_rf,  mae_rf  = report("RandomForest", y_test, pred_rf)

#6) Plots: predicted vs actual + residuals 
plt.figure(figsize=(7,5))
sns.scatterplot(x=y_test, y=pred_rf)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "k--", alpha=0.5)
plt.xlabel("Actual"); plt.ylabel("Predicted")
plt.title(f"{TARGET} — RF predictions (test)")
plt.tight_layout(); plt.savefig(OUT_DIR / "rf_pred_vs_actual.png", dpi=150); plt.show()

plt.figure(figsize=(7,5))
resid = y_test - pred_rf
sns.histplot(resid, bins=15, kde=True)
plt.title("Random Forest residuals (test)")
plt.xlabel("Residual (Actual - Predicted)")
plt.tight_layout(); plt.savefig(OUT_DIR / "rf_residuals_hist.png", dpi=150); plt.show()

#  7) Permutation importance 
perm = permutation_importance(rf, X_test, y_test, n_repeats=30, random_state=42, n_jobs=-1)
imp = (pd.DataFrame({"feature": X_train.columns, "importance": perm.importances_mean})
         .sort_values("importance", ascending=False))

top20 = imp.head(20)
plt.figure(figsize=(8,6))
sns.barplot(data=top20, y="feature", x="importance")
plt.title("Permutation Importance (RF) — top 20")
plt.tight_layout(); plt.savefig(OUT_DIR / "rf_perm_importance.png", dpi=150); plt.show()

print("\nTop drivers:")
display(top20.head(10))

#  8)  “what-if”: reduce the top driver by 10% 
if not top20.empty:
    main_driver = top20.iloc[0]["feature"]
    print(f"\nWhat-if: reduce '{main_driver}' by 10% on test rows …")
    X_sim = X_test.copy()
    X_sim[main_driver] = X_sim[main_driver] * 0.90
    pred_sim = rf.predict(X_sim)

    # Negative delta = predicted improvement (because lower is better for a mortality rate)
    delta = pred_sim - pred_rf
    by_state = (pd.DataFrame({"State": y_test.index.get_level_values(0), "Delta": delta})
                  .groupby("State", as_index=False)["Delta"].mean()
                  .sort_values("Delta"))

    plt.figure(figsize=(9,5))
    sns.barplot(data=by_state, x="State", y="Delta")
    plt.axhline(0, color="k", lw=1)
    plt.title(f"What-if: 10% drop in '{main_driver}' — change in predicted {TARGET}")
    plt.ylabel("Change in predicted rate")
    plt.tight_layout(); plt.savefig(OUT_DIR / "rf_what_if_driver_minus10.png", dpi=150); plt.show()

#  9) one-step “forecast” demo 
# Train on all but the latest year; predict the next (latest+1) for each state using lag features.
latest_year = wide_lag.index.get_level_values("Year").max()
last_rows   = wide_lag.xs(latest_year, level="Year", drop_level=False)
last_feats  = last_rows[X_train.columns].fillna(col_means)
next_pred   = rf.predict(last_feats)

forecast = last_rows.reset_index()[["State", "Year"]].copy()
forecast["Forecast_Year"] = forecast["Year"] + 1
forecast[f"Predicted_{TARGET}"] = next_pred

print("\nSample one-step forecast (per state/region):")
display(forecast.head(20))

forecast.to_csv(OUT_DIR / "rf_one_step_forecast_next_year.csv", index=False)

print("\nSaved files in:", OUT_DIR.resolve())
for f in ["rf_pred_vs_actual.png",
          "rf_residuals_hist.png",
          "rf_perm_importance.png",
          "rf_what_if_driver_minus10.png",
          "rf_one_step_forecast_next_year.csv"]:
    print(" -", OUT_DIR / f)
